In [1]:
import hashlib
import os
import tarfile
import zipfile
import requests

维护一个字典DATA_HUB，其将数据集名称的字符串映射到数据集相关的二元组上，这个二元组包含数据集的url和验证文件完整性的sha-1秘钥。

In [2]:
DATA_HUB = dict()
DATA_URL = 'http://d2l-data.s3-accelerate.amazonaws.com/'

用download函数来下载数据集，将数据集缓存在本地目录中，并返回下载的文件名称。如果缓存目录中已经存在次数据集，并且其sha-1与存储在DATA_HUB中的相匹配，我们将使用缓存的文件以避免重复下载。

In [3]:
def download(name, cache_dir=os.path.join('..', 'data')):   #@save
    """ 下载一个DATA_HUB中的文件并返回本地文件名 """
    assert name in DATA_HUB, f"{name}不存在于{DATA_HUB}"
    url, sha1_hash = DATA_HUB[name]
    os.makedirs(cache_dir, exist_ok=True)
    fname = os.path.join(cache_dir, url.split('/')[-1])
    if os.path.exists(fname):
        sha1 = hashlib.sha1()
        with open(fname, 'rb') as f:
            while True:
                data = f.read(1048576)
                if not data:
                    break
                sha1.update(data)
        if sha1.hexdigest() == sha1_hash:
            return fname
    print(f'正在从{url}下载{fname}...')
    r = requests.get(url, stream=True, verify=True)
    with open(fname, 'wb') as f:
        f.write(r.content)
    return fname


def download_extract(name, folder=None):  #@save
    """下载并解压zip/tar文件。"""
    fname = download(name)
    base_dir = os.path.dirname(fname)
    data_dir, ext = os.path.splitext(fname)
    if ext == '.zip':
        fp = zipfile.ZipFile(fname, 'r')
    elif ext in ('.tar', '.gz'):
        fp = tarfile.open(fname, 'r')
    else:
        assert False, '只有zip/tar文件可以被解压缩。'
    fp.extractall(base_dir)
    return os.path.join(base_dir, folder) if folder else data_dir


def download_all():  #@save
    """下载DATA_HUB中的所有文件。"""
    for name in DATA_HUB:
        download(name)

竞赛数据分为训练集合测试集，每条记录都包括房屋的属性值和属性，如街道类型、施工年份、屋顶类型、地下室状况等，一些数据也有可能已经丢失，丢失值会被标记为"NA"，因此开始训练之前要清理数据。

In [22]:
import numpy as np
import pandas as pd
import torch
from torch.utils import data
from torch import nn
from d2l import torch as d2l

DATA_HUB['kaggle_house_train'] = (DATA_URL + 'kaggle_house_pred_train.csv',
                                  '585e9cc93e70b39160e7921475f9bcd7d31219ce')

DATA_HUB['kaggle_house_test'] = (DATA_URL + 'kaggle_house_pred_test.csv',
                                 'fa19780a7b011d9b009e8bff8e99922a8ee2eb90')

train_data = pd.read_csv(download('kaggle_house_train'))
test_data = pd.read_csv(download('kaggle_house_test'))

In [5]:
print(train_data.shape)
print(test_data.shape)

(1460, 81)
(1459, 80)


看看前四个和最后两个特征，以及相应标签（房价）

In [6]:
print(train_data.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]])

   Id  MSSubClass MSZoning  LotFrontage SaleType SaleCondition  SalePrice
0   1          60       RL         65.0       WD        Normal     208500
1   2          20       RL         80.0       WD        Normal     181500
2   3          60       RL         68.0       WD        Normal     223500
3   4          70       RL         60.0       WD       Abnorml     140000


In [7]:
# 整合所有特征
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))
print(all_features.shape)
print(all_features.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]])

(2919, 79)
   MSSubClass MSZoning  LotFrontage  LotArea  YrSold SaleType SaleCondition
0          60       RL         65.0     8450    2008       WD        Normal
1          20       RL         80.0     9600    2007       WD        Normal
2          60       RL         68.0    11250    2008       WD        Normal
3          70       RL         60.0     9550    2006       WD       Abnorml


## 数据清洗

先来处理非文本特征

将缺失值替换为相应特征的平均值

为了将所有特征放在一个共同的尺度上，我们通过将特征重新缩放到零均值和单位方差来标准化数据

$$
 x ← \frac{x-μ}{σ}
$$

这样特征值就具有了零均值和单位方差

In [8]:
numeric_features = all_features.dtypes[
    all_features.dtypes != 'object'].index  # 将数值特征挑出来
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 在标准化数据后，所有数据意味着消失，因此我们可以将缺失值设为0
all_features[numeric_features] = all_features[numeric_features].fillna(0)

接下来处理文本特征值，用一次独热编码替换。

例如"MSZoning"包含值"RL"和"Rm"，可以创建两个新的特征"MSZoning_RL"和"MSZoning_Rm"来替换原来的特征标签。

In [9]:
all_features = pd.get_dummies(all_features, dummy_na=True)  # 将"NA"视为有效特征值，并为其创建指示符特征
all_features.shape

(2919, 331)

最后从pandas格式提取Numpy格式，并将其转换为张量进行训练

In [10]:
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values,
                              dtype=torch.float32)
test_features = torch.tensor(all_features[n_train:].values,
                             dtype=torch.float32)
train_labels = torch.tensor(train_data.SalePrice.values.reshape(-1, 1),
                            dtype=torch.float32)

# 训练

In [11]:
loss = nn.MSELoss()
in_features = train_features.shape[1]   # 331

def get_net():
    net = nn.Sequential(nn.Linear(in_features,1))
    return net

对于房价，我们关心的是相对数量而不是绝对数量，因此我们更关心相对误差$$\frac{y-\hat{y}}{y}$$

因此loss使用均方根误差
$$\sqrt{\frac{1}{n}\sum_{i=1}^n\left(\log y_i -\log \hat{y}_i\right)^2}$$

In [12]:
def log_rmse(net, features, labels):
    # 为了在取对数时进一步稳定该值，将小于1的值设为1
    clipped_preds = torch.clamp(net(features), 1, float('inf'))     # 将值束缚在[1,inf]之间，小于1的设为1，大于inf的设为inf
    rmse = torch.sqrt(loss(torch.log(clipped_preds), torch.log(labels)))
    return rmse.item()

In [23]:
def load_array(data_arrays, batch_size, is_train=True):
      """ 构造一个基于Pytorch的数据迭代器 """
      dataset = data.TensorDataset(*data_arrays)  # 表明datat_attays是一个元组
      return data.DataLoader(dataset, batch_size, shuffle=is_train)  # is_train表示是否希望数据迭代器对象在每个迭代周期内打乱数据


def train(net, train_features, train_labels, test_features, test_labels,
      num_epochs, learning_rate, weight_decay, batch_size):
      train_ls, test_ls = [], []
      train_iter = load_array((train_features, train_labels), batch_size)
      optimizer = torch.optim.Adam(net.parameters(),
                                    lr=learning_rate,
                                    weight_decay=weight_decay)  # 使用Adam优化算法
      for epoch in range(num_epochs):
            for X, y in train_iter:
                  optimizer.zero_grad()
                  l = loss(net(X), y)
                  l.backward()
                  optimizer.step()
            train_ls.append(log_rmse(net, train_features, train_labels))
            if test_ls is not None:
                  test_ls.append(log_rmse(net, test_features, test_labels))
      return train_ls, test_ls

### K折交叉验证

在K折交叉验证时返回第i折的数据，选择第i个切片作为验证数据，其余部分作为训练数据。

In [14]:
def get_k_fold_data(k, i, X, y):
    """ 当返回到第i折时，返回训练用数据和验证用的数据 """
    assert k > 1
    fold_size = X.shape[0] // k  # 返回商的整数部分，向下取整
    X_train, y_train = None, None
    for j in range(k):
        idx = slice(j * fold_size, (j + 1) * fold_size)
        X_part, y_part = X[idx, :], y[idx]
        if j == i:
            X_valid, y_valid = X_part, y_part
        elif X_train is None:
            X_train, y_train = X_part, y_part
        else:
            X_train = torch.cat([X_train, X_part], 0)
            y_train = torch.cat([y_train, y_part], 0)
    return X_train, y_train, X_valid, y_valid

In [26]:
def k_fold(k, X_train, y_train, num_epochs, learning_rate, weight_decay,
            batch_size):
      """ 在交叉验证中训练k次后返回训练和验证的平均误差 """
      train_l_sum, valid_l_sum = 0, 0
      for i in range(k):
            data = get_k_fold_data(k, i, X_train, y_train)
            net = get_net()
            train_ls, valid_ls = train(net, *data, num_epochs, learning_rate,
                                          weight_decay, batch_size)
            train_l_sum += train_ls[-1]
            valid_l_sum += valid_ls[-1]
            print(f'fold {i + 1}, train log rmse {float(train_ls[-1]):f}, '
                  f'valid log rmse {float(valid_ls[-1]):f}')
      return train_l_sum / k, valid_l_sum / k

In [27]:
k = 5
num_epochs = 100
lr = 5
weight_decay = 0
batch_size = 64
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr,
                            weight_decay, batch_size)
print(f'{k}-折验证: 平均训练log rmse: {float(train_l):f}, '
        f'平均验证log rmse: {float(valid_l):f}')


gg
fold 1, train log rmse 0.169715, valid log rmse 0.156277
5-折验证: 平均训练log rmse: 0.033943, 平均验证log rmse: 0.031255
